# Interschulz Product & Company Crawler

In [1]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time

chromedriver = 'C:\Code\Webdriver\chromedriver.exe'
driver = webdriver.Chrome(chromedriver)
action = ActionChains(driver)
# 쿠키 없이 들어가면 아무정보도 안뜸
driver.get('https://www.interschutz.de/en/')
time.sleep(1)

driver.get('https://www.interschutz.de/en/search/?category=ep')

for i in range(202):
    bottom = driver.find_element(By.CSS_SELECTOR,"#app > div.flex-container.flex-dir-column.fullheight > div.flex-child-shrink > section.c.footer.is-dark.module-theme-400")
    action.move_to_element(bottom).perform()
    driver.find_element(By.XPATH,'//*[@id="search-results-reveal"]/div/div[1]/div/div/div[2]/section/div/div[2]/div[2]/section/a[6]/i').click()

time.sleep(1)

time.sleep(5)
driver.quit()

C:\Users\user\AppData\Local\Temp\ipykernel_2504\219171253.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


AttributeError: move_to requires a WebElement

In [4]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

# 속도 향상을 위한 이미지 로딩 제거
chromedriver = 'C:\Code\Webdriver\chromedriver.exe'
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)

action = ActionChains(driver)
# 쿠키 없이 들어가면 아무정보도 안뜸
driver.get('https://www.interschutz.de/en/')
time.sleep(1)

driver.get('https://www.interschutz.de/en/search/?category=ep')
# 마우스 오버
#action = ActionChains(driver)
#action.move_to_element(driver.find_element_by_xpath('//*[@id="app"]/div[1]/div[3]/div/nav/div[3]/div[1]/button[2]')).perform()
#driver.find_element_by_xpath('//*[@id="app"]/div[1]/div[3]/div/div/div/a[5]').click()
soup_list = []
for i in range(202-1):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    #result = soup.find_all('o search snippet module-theme-100e')
    #print(soup.text)
    # Soup는 다 끝나고 한방에 로딩
    soup_list.append(soup)
    bottom = driver.find_element(By.CSS_SELECTOR,"#app > div.flex-container.flex-dir-column.fullheight > div.flex-child-shrink > section.c.footer.is-dark.module-theme-400")
    action.move_to_element(bottom).perform()
    #driver.find_element(By.XPATH,'/html/body/main/div[2]/div[1]/section[1]/form/section[2]/div/div[1]/div/div/div[2]/section/div/div[2]/div[2]/section/a[6]').click()

    # Xpath 찾는속도 느려서 그냥 CSS SELECTOR 처리
    try:
        driver.find_element(By.CSS_SELECTOR,'#search-results-reveal > div > div.reveal-master > div > div > div.cell.search-results-list.medium-8 > section > div > div.grid-x.search-snippet-list-footer > div:nth-child(2) > section > a:nth-child(8)').click()
    except:
        driver.find_element(By.CSS_SELECTOR,'#search-results-reveal > div > div.reveal-master > div > div > div.cell.search-results-list.medium-8 > section > div > div.grid-x.search-snippet-list-footer > div:nth-child(2) > section > a:nth-child(7)').click()
    time.sleep(1)

driver.quit()

C:\Users\user\AppData\Local\Temp\ipykernel_2504\3052831934.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
C:\Users\user\AppData\Local\Temp\ipykernel_2504\3052831934.py:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)


In [5]:
link_list = []
for i in soup_list:
    temp_list = i.find_all('a','o link as-block fx dropshadow for-child')
    for l in temp_list:
        link_list.append(l.attrs['href'])

In [6]:
import requests
import pandas as pd
from tqdm import tqdm

prd_list = [] # 제품만 긁어도 됨

base_url = "https://www.interschutz.de/"

for i in tqdm(link_list):
    switch = i.split("/")[1]
    response = requests.get(base_url+i)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    if switch == 'product': #제품일 경우
        #print("제품 :",len(prd_list)+1)
        soup.append(prd_list)
    else:
        pass


  3%|▎         | 114/4020 [01:43<58:55,  1.10it/s] 


KeyboardInterrupt: 

In [14]:
prd_link_list = []
for i in link_list:
    switch = i.split("/")[1]
    if switch == 'product':
        prd_link_list.append(i)

In [16]:
import requests
import pandas as pd
from tqdm import tqdm

prd_soup_list = [] # 제품만 긁어도 됨

base_url = "https://www.interschutz.de/"

for i in tqdm(prd_link_list):
    switch = i.split("/")[1]
    response = requests.get(base_url+i)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    prd_soup_list.append(soup)

100%|██████████| 2668/2668 [30:32<00:00,  1.46it/s]  


In [69]:
for i in prd_soup_list[:2]:
    #print(i)
    temp = i.select_one("#tabs-top")
    print(temp.select_one('h2'))
    print(temp.select('li'))
    #print(i.select_one("#app > div.flex-container.flex-dir-column.fullheight > div.flex-child-grow > div > header > div > div > div.cell.small-12.page-intro-content.medium-10.large-9 > div.page-intro-description.with-function-bar > div.grid-x.grid-padding-x > div.cell.auto > div:nth-child(1)"))
    #print(i.select("#app > div.flex-container.flex-dir-column.fullheight > div.flex-child-grow > div > header > div > div > div.cell.small-12.page-intro-content.medium-10.large-9 > div.t.set-800-bold > h1"))

<h2 class="t set-700-bold as-headline">Contact</h2>
[<li>Huangjiabu, Yuyao -</li>, <li>315466 Ningbo</li>, <li>China</li>, <li>
<o-link href="http://www.fuhua-valve.com" underline="">Website</o-link>
</li>, <li>Phone: +86 574 62017181</li>, <li>Fax: +86 574 62016923</li>]
<h2 class="t set-250-bold as-headline">Otto Ganter GmbH &amp; Co. KG Normteilefabrik</h2>
[<li>Triberger Str. 3</li>, <li>78120 Furtwangen</li>, <li>Germany</li>, <li>
<o-link href="http://www.ganternorm.com" underline="">Website</o-link>
</li>, <li>Phone: +49 7723 65070</li>, <li>Fax: +49 7723 4659</li>, <li>Brunn am Gebirge</li>, <li>AT</li>, <li>Soraluze </li>, <li>ES</li>, <li>Piaseczno</li>, <li>PL</li>, <li>Shanghai</li>, <li>CN</li>, <li>Istanbul</li>, <li>TR</li>, <li>Prag</li>, <li>CZ</li>, <li>Noida</li>, <li>IN</li>, <li>New Berlin</li>, <li>US</li>]


In [77]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
# 속도 향상을 위한 이미지 로딩 제거
chromedriver = 'C:\Code\Webdriver\chromedriver.exe'
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
action = ActionChains(driver)

v1_list = []
v2_list = []
v3_list = []
v4_list = []
v5_list = []

base_url = "https://www.interschutz.de/"

for i in tqdm(prd_link_list):
    url = base_url+i
    # URL 접속
    driver.get(url)

    # 제품명
    v1 = driver.find_element(By.CSS_SELECTOR,"#app > div.flex-container.flex-dir-column.fullheight > div.flex-child-grow > div > header > div > div > div.cell.small-12.page-intro-content.medium-10.large-9 > div.t.set-800-bold > h1").text
    # 제조사
    v2 = driver.find_element(By.CSS_SELECTOR,"#app > div.flex-container.flex-dir-column.fullheight > div.flex-child-grow > div > header > div > div > div.cell.small-12.page-intro-content.medium-10.large-9 > div.page-intro-description.with-function-bar > div.grid-x.grid-padding-x > div.cell.auto > div:nth-child(1)").text
    # 설명
    v3 = driver.find_element(By.CSS_SELECTOR,"#detailProductId1 > div:nth-child(2) > div > div > div > div > p").text
    # 제품그룹
    try:
        v4 = driver.find_element(By.CSS_SELECTOR,"#detailProductId1 > div:nth-child(2) > div > div > div > div > ul").text.split("\n")
    except:
        v4.append([])
    # 제조사명
    v5 = driver.find_element(By.CSS_SELECTOR,"#tabs-top > div.tabs-track > div > div.is-active > div > div > div:nth-child(1) > h2").text

    v1_list.append(v1)
    v2_list.append(v2)
    v3_list.append(v3)
    v4_list.append(v4)
    v5_list.append(v5)


C:\Users\user\AppData\Local\Temp\ipykernel_2504\2980603150.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
C:\Users\user\AppData\Local\Temp\ipykernel_2504\2980603150.py:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
  1%|▏         | 39/2668 [01:31<1:42:50,  2.35s/it]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#detailProductId1 > div:nth-child(2) > div > div > div > div > p"}
  (Session info: chrome=103.0.5060.134)
Stacktrace:
Backtrace:
	Ordinal0 [0x008E5FD3+2187219]
	Ordinal0 [0x0087E6D1+1763025]
	Ordinal0 [0x00793E78+802424]
	Ordinal0 [0x007C1C10+990224]
	Ordinal0 [0x007C1EAB+990891]
	Ordinal0 [0x007EEC92+1174674]
	Ordinal0 [0x007DCBD4+1100756]
	Ordinal0 [0x007ECFC2+1167298]
	Ordinal0 [0x007DC9A6+1100198]
	Ordinal0 [0x007B6F80+946048]
	Ordinal0 [0x007B7E76+949878]
	GetHandleVerifier [0x00B890C2+2721218]
	GetHandleVerifier [0x00B7AAF0+2662384]
	GetHandleVerifier [0x0097137A+526458]
	GetHandleVerifier [0x00970416+522518]
	Ordinal0 [0x00884EAB+1789611]
	Ordinal0 [0x008897A8+1808296]
	Ordinal0 [0x00889895+1808533]
	Ordinal0 [0x008926C1+1844929]
	BaseThreadInitThunk [0x772EFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77437A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77437A6E+238

In [80]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
# 속도 향상을 위한 이미지 로딩 제거
chromedriver = 'C:\Code\Webdriver\chromedriver.exe'
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
action = ActionChains(driver)

v1_list = []
v2_list = []
v3_list = []
v4_list = []
v5_list = []
v6_list = []

base_url = "https://www.interschutz.de/"

for i in tqdm(prd_link_list):
    url = base_url+i
    # URL 접속
    driver.get(url)

    # 제품명
    try:
        v1 = driver.find_element(By.CSS_SELECTOR,"#app > div.flex-container.flex-dir-column.fullheight > div.flex-child-grow > div > header > div > div > div.cell.small-12.page-intro-content.medium-10.large-9 > div.t.set-800-bold > h1").text
    except:
        v1 = np.nan
    # 제조사
    try:
        v2 = driver.find_element(By.CSS_SELECTOR,"#app > div.flex-container.flex-dir-column.fullheight > div.flex-child-grow > div > header > div > div > div.cell.small-12.page-intro-content.medium-10.large-9 > div.page-intro-description.with-function-bar > div.grid-x.grid-padding-x > div.cell.auto > div:nth-child(1)").text
    except:
        v2 = np.nan
    # 설명
    try:
        try:
            v3 = driver.find_element(By.CSS_SELECTOR,"#detailProductId1 > div:nth-child(2) > div > div > div > div > p").text
        except:
            v3 = driver.find_element(By.CSS_SELECTOR,"#detailProductId1 > div:nth-child(3) > div > div > div > div > p").text
    except:
        v3 = np.nan
    # 제품그룹
    try:
        v4 = driver.find_element(By.CSS_SELECTOR,"#detailProductId1 > div:nth-child(3) > div > div > div > div > ul").text.split("\n")
    except:
        try:
            v4 = driver.find_element(By.CSS_SELECTOR,"#detailProductId1 > div:nth-child(2) > div > div > div > div > ul").text.split("\n")
        except:
            v4 = np.nan

    # 제조사명
    v5 = driver.find_element(By.CSS_SELECTOR,"#tabs-top > div.tabs-track > div > div.is-active > div > div > div:nth-child(1) > h2").text
    try:
        v6 = driver.find_element(By.CSS_SELECTOR,"#detailProductId1 > div:nth-child(3) > div > div > div > div > a").get_attribute('href')
    except:
        v6 = np.nan

    v1_list.append(v1)
    v2_list.append(v2)
    v3_list.append(v3)
    v4_list.append(v4)
    v5_list.append(v5)
    v6_list.append(v6)

C:\Users\user\AppData\Local\Temp\ipykernel_2504\3933938798.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
C:\Users\user\AppData\Local\Temp\ipykernel_2504\3933938798.py:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
100%|██████████| 2668/2668 [1:42:29<00:00,  2.30s/it]


In [82]:
df = pd.DataFrame({"v1":v1_list,
            "v2":v2_list,
            "v3":v3_list,
            "v4":v4_list,
            "v5":v5_list,
            "v6":v6_list,
            "v7":prd_link_list})

In [84]:
df.to_csv("prd_crawl.csv")